# multipass
> Multipass VM management for local Linux testing

In [ ]:
#| default_exp multipass

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os, json, subprocess, tempfile
from pathlib import Path
from functools import partial
from fastcore.all import concat

## CLI wrapper

`callmultipass()` mirrors `calldocker()` — runs the multipass CLI and returns stdout. `Multipass` uses the same kwargs-to-flags convention as `Docker`.

In [ ]:
#| export
def callmultipass(*args):
    'Run a multipass CLI command, return stdout.'
    return subprocess.run(('multipass',) + args, capture_output=True, text=True, check=True).stdout.strip()

class Multipass:
    'Wrap multipass CLI: __getattr__ dispatches subcommands, kwargs become flags'
    def __call__(self, cmd, *args, **kwargs):
        fargs = list(args)
        fargs += concat([f'-{k}', str(v)] for k,v in kwargs.items() if len(k)==1 and v not in (True, False, None))
        fargs += [f'-{k}' for k,v in kwargs.items() if len(k)==1 and v is True]
        fargs += [f'--{k.rstrip("_").replace("_","-")}={v}' for k,v in kwargs.items() if len(k)>1 and v not in (True, False, None)]
        fargs += [f'--{k.rstrip("_").replace("_","-")}' for k,v in kwargs.items() if len(k)>1 and v is True]
        return callmultipass(cmd, *fargs)

    def __getattr__(self, nm):
        if nm.startswith('_'): raise AttributeError(nm)
        return partial(self, nm.replace('_', '-'))

mp = Multipass()

## cloud_init_yaml

Generates a `#cloud-config` YAML string for Multipass `--cloud-init`. When `docker=True` (default) it installs Docker via `get.docker.com`.

In [ ]:
#| export
def cloud_init_yaml(docker=True, packages=None, cmds=None) -> str:
    'Generate cloud-init YAML for a Multipass VM'
    pkgs = ['curl'] + list(packages or [])
    lines = ['#cloud-config', 'package_update: true', 'packages:']
    for p in pkgs:
        lines.append(f'  - {p}')
    rcmds = []
    if docker:
        rcmds += [
            'curl -fsSL https://get.docker.com | sh',
            'usermod -aG docker ubuntu',
            'systemctl enable --now docker',
        ]
    rcmds += list(cmds or [])
    if rcmds:
        lines.append('runcmd:')
        for c in rcmds:
            lines.append(f'  - {c}')
    return '\n'.join(lines) + '\n'

In [ ]:
init = cloud_init_yaml()
assert '#cloud-config' in init
assert 'get.docker.com' in init
assert 'usermod' in init
print('cloud_init_yaml() default OK')

init2 = cloud_init_yaml(docker=False, packages=['git', 'vim'], cmds=['echo hello'])
assert 'get.docker.com' not in init2
assert '  - git' in init2
assert 'echo hello' in init2
print('cloud_init_yaml() custom OK')

print(init)

## VM helpers

In [ ]:
#| export
def launch(name, image='22.04', cpus=1, memory='1G', disk='10G', cloud_init=None, mounts=None):
    'Launch a Multipass VM. cloud_init can be YAML string or path to existing file.'
    args = ['--image', image, '--cpus', str(cpus), '--memory', memory, '--disk', disk]
    tmp_path = None
    if cloud_init is not None:
        if os.path.isfile(cloud_init):
            args += ['--cloud-init', cloud_init]
        else:
            fd, tmp_path = tempfile.mkstemp(suffix='.yaml')
            with os.fdopen(fd, 'w') as f:
                f.write(cloud_init)
            args += ['--cloud-init', tmp_path]
    for hp, vp in (mounts or {}).items():
        args += ['--mount', f'{hp}:{vp}']
    try:
        callmultipass('launch', name, *args)
    finally:
        if tmp_path is not None:
            Path(tmp_path).unlink(missing_ok=True)
    return name

In [ ]:
#| export
def vms(running=False) -> list:
    'List Multipass VM names. running=True filters to Running state.'
    data = json.loads(callmultipass('list', '--format', 'json'))
    lst = data.get('list', [])
    if running: lst = [v for v in lst if v.get('state') == 'Running']
    return [v['name'] for v in lst]

def vm_ip(name) -> str:
    'Get the IPv4 address of a Multipass VM.'
    data = json.loads(callmultipass('info', name, '--format', 'json'))
    return data['info'][name]['ipv4'][0]

def exec_(name, *cmd) -> str:
    'Run a command in a Multipass VM.'
    return callmultipass('exec', name, '--', *cmd)

def delete(name, purge=True) -> None:
    'Delete a Multipass VM.'
    mp.delete(name)
    if purge: mp.purge()

def transfer(src, dst) -> None:
    'Transfer files to/from a Multipass VM. Use "vmname:/path" for VM paths.'
    callmultipass('transfer', src, dst)

In [ ]:
# Test vms() - runs without error
try:
    result = vms()
    assert isinstance(result, list)
    print(f'vms() OK: {result}')
except Exception as e:
    print(f'multipass not available: {e}')

# Test vm_ip() error path on a non-existent VM
try:
    vm_ip('nonexistent-vm-dockr-test')
    print('ERROR: should have raised')
except subprocess.CalledProcessError:
    print('vm_ip() error path OK')
except Exception as e:
    print(f'Other error (multipass not installed?): {e}')

In [ ]:
#| export
def launch_docker_vm(name, image='22.04', cpus=2, memory='2G', disk='20G',
                     packages=None, mounts=None) -> str:
    'Launch a Multipass VM with Docker pre-installed. Convenience wrapper for cloud_init_yaml + launch.'
    return launch(name, image=image, cpus=cpus, memory=memory, disk=disk,
                  cloud_init=cloud_init_yaml(docker=True, packages=packages),
                  mounts=mounts)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()